<a href="https://colab.research.google.com/github/Multi-Objective-NAS/invertible-nas-algorithm/blob/master/Neural_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [1]:
pip install git+https://github.com/rtqichen/torchdiffeq

  Cloning https://github.com/rtqichen/torchdiffeq to /tmp/pip-req-build-21ij5ct8
  Running command git clone -q https://github.com/rtqichen/torchdiffeq /tmp/pip-req-build-21ij5ct8
  Created wheel for torchdiffeq: filename=torchdiffeq-0.0.1-cp36-none-any.whl size=27578 sha256=72485f28855d2b14e3b5c7d77fa61b78dc0b5004a12e50f20e73bee211c7f671
  Stored in directory: /tmp/pip-ephem-wheel-cache-erwso6_6/wheels/3f/76/69/01867bf3355c3bc8bae7e556b17b44c395b6cda5e76fd8ddc7
Successfully built torchdiffeq


In [2]:
!git clone https://github.com/rtqichen/torchdiffeq

Cloning into 'torchdiffeq'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 229 (delta 3), reused 4 (delta 1), pack-reused 214
Receiving objects: 100% (229/229), 715.37 KiB | 5.42 MiB/s, done.
Resolving deltas: 100% (100/100), done.


#Installation dataset

In [3]:
#!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord
!curl -O https://storage.googleapis.com/nasbench/nasbench_only108.tfrecord
# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
#!pip install ./nasbench

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498M  100  498M    0     0  33.2M      0  0:00:15  0:00:15 --:--:-- 42.8M
Cloning into 'nasbench'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 96 (delta 0), reused 2 (delta 0), pack-reused 92
Unpacking objects: 100% (96/96), done.


#Setting

In [0]:
import os
import argparse
import logging
import time
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [0]:
from torchdiffeq import odeint_adjoint as odeint

###Instatiate the summary writers


In [6]:
'''
# Install latest Tensorflow build
!pip install -q tf-nightly-2.0-preview

from tensorflow import summary
%load_ext tensorboard.notebook

current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)
'''

"\n# Install latest Tensorflow build\n!pip install -q tf-nightly-2.0-preview\n\nfrom tensorflow import summary\n%load_ext tensorboard.notebook\n\ncurrent_time = str(datetime.datetime.now().timestamp())\ntrain_log_dir = 'logs/tensorboard/train/' + current_time\ntest_log_dir = 'logs/tensorboard/test/' + current_time\ntrain_summary_writer = summary.create_file_writer(train_log_dir)\ntest_summary_writer = summary.create_file_writer(test_log_dir)\n"

### Constants

In [0]:
'''
Main Argument
'''
DEVICE = torch.device('cuda:0')
#DEVICE = torch.device('cpu')
BATCH_SIZE = 50
LR = 0.1
NEPOCHS = 1 
RTOL = 1e-7
ATOL = 1e-9
EDGE_NUM = 9

'''
Data Set
'''
# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3, OUTPUT]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix
EMBED_SIZE = 7 + 7 + len(ALLOWED_OPS)

In [8]:
%cd nasbench
%ls
!pwd

/content/nasbench
CONTRIBUTING.md  images/  nasbench/       README.md
example.py       LICENSE  NASBench.ipynb  setup.py
/content/nasbench


In [9]:
from nasbench import api
#nasbench = api.NASBench('nasbench_full.tfrecord')

%cd ..

nasbench_api = api.NASBench('nasbench_only108.tfrecord')
nasbench_hashkeys = nasbench_api.hash_iterator()




/content
Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 45 seconds


#Code

In [0]:
def encode(mat, ops):
    def _sanity_check(idx1, opidx, idx2):
        if idx1 not in range(7):
            return False
        if idx2 not in range(7):
            return False
        if opidx not in range(4):
            return False
        return True

    encoded = []
    for inbound, outbound in zip(*np.array(mat).nonzero()):
        op = ALLOWED_OPS.index(ops[outbound])
        assert _sanity_check(inbound, op, outbound)

        embed = [0] * EMBED_SIZE
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1

        encoded.append(embed)

    assert np.array(encoded).shape == (9, EMBED_SIZE)
    return encoded


def max_index(output):
    # (1, 9,18)
    encoded = []
    outputs = [output_ for output_ in output[0]]
    
    for output in outputs:
        embed = [0 for _ in range(EMBED_SIZE)]
        inbound, op, outbound = np.argmax(output[:7]), np.argmax(output[7:-7]), np.argmax(output[-7:])
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1
        assert np.sum(np.array(embed)) == 3
        encoded.append(embed)

    assert np.array(encoded).shape == (9, EMBED_SIZE)
    return encoded


def decode(encoded):
    mat = np.zeros((7, 7))
    ops = [INPUT] + [CONV3X3 for _ in range(5)] + [OUTPUT]

    for embed in encoded:
        inbound, op, outbound = np.nonzero(np.array(embed))[0]
        op -= 7
        outbound -= (7 + len(ALLOWED_OPS))
        ops[outbound] = ALLOWED_OPS[op]
        if inbound > outbound:
            (inbound, outbound) = (outbound, inbound)
        elif inbound == outbound:
            continue
        mat[inbound][outbound] = 1

    return mat.tolist(), ops

In [0]:
assert np.array([[1,2,3],[1,2,3]]).shape == (2,3)

In [0]:
class DataLoader(object):
    def __init__(self, start, end, batchsize=None):
        self.start = start
        self.end = end
        self.cur = start
        if batchsize is None:
            self.batchsize = end - start
        else:
            self.batchsize = batchsize
        self.size = end - start

    def __run_experiment(self, mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench_api.query(model_spec)
        return q['validation_accuracy'], q['training_time']

    def next(self):
        global nasbench_hashkeys

        xset = []
        yset = []
        if self.cur == self.end:
            self.cur = self.end
        for idx in range(self.cur, min(self.cur + self.batchsize, self.end)):
            hash_val = nasbench_hashkeys[idx]
            fixed_stat, _ = nasbench_api.get_metrics_from_hash(hash_val)
            mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
            x = encode(mat, op)
            y = self.__run_experiment(mat, op)
            xset.append(x)
            yset.append(y)
        xset = torch.FloatTensor(xset)
        yset = torch.FloatTensor(yset)
        return xset, yset


def train_val_test_split(batch_size=100, validation_ratio=0.1, test_ratio=0.1):
    global nasbench_hashkeys

    revised = []
    for hash_val in nasbench_hashkeys:
        fixed_stat, _ = nasbench_api.get_metrics_from_hash(hash_val)
        mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
        if np.sum(mat) == EDGE_NUM:
            revised.append(hash_val)

    total_size = (len(revised) // batch_size) * batch_size
    nasbench_hashkeys = revised[: total_size]
    val_size = int(total_size * validation_ratio)
    test_size = int(total_size * test_ratio)

    return DataLoader(val_size+test_size, total_size, batch_size), DataLoader(0, val_size), DataLoader(val_size, val_size+test_size)

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm( dim, dim)

In [0]:
class ConcatConv1d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(ConcatConv1d, self).__init__()
        self._layer = nn.Conv1d(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv1d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out

In [15]:
#test
a = torch.ones(3, 2)
b = torch.zeros(3, 3)
torch.cat([a, b], dim=1)

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

In [0]:
class ODEBlock(nn.Module):

    def __init__(self, odefunc, method):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()
        self.method = method

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        #batch_t = T_series[:x.shape[0]]
        out = odeint(self.odefunc, x, self.integration_time , rtol=RTOL, atol=ATOL, method = self.method)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value

In [0]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        ret = x.view(-1, shape)
        return ret


In [0]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


In [0]:
def learning_rate_with_decay(batch_size, batch_denom, batches_per_epoch, boundary_epochs, decay_rates):
    initial_learning_rate = LR * batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn

In [0]:
def accuracy(model, dataloader):
    datasize = dataloader.end - dataloader.start
    x, y = dataloader.next(datasize)
    x = torch.FloatTensor(x)
    x = x.to(DEVICE)
    y = y.numpy()
    result = model(x).cpu().detach().numpy()
    total_correct = np.sum(np.square(result - y))
    return total_correct / datasize

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
class ODEModel(nn.Module):
    def __init__(self, dim, method=None):
        # dim is edge number
        super(ODEModel, self).__init__()
        self.feature_layers = [ODEBlock(ODEfunc(dim), method)]
        self.fc_layers = [norm(dim), nn.ReLU(inplace=True), nn.AdaptiveAvgPool1d(1), Flatten(), nn.Linear(dim, 2)]
        self.model = nn.Sequential(*self.feature_layers, *self.fc_layers).to(DEVICE)

    def forward(self, x):
        return self.model(x)


class Trainer():
    def __init__(self, model, optimizer, criterion, accuracy, lr_fn, meter_fn):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.accuracy = accuracy
        self.lr_fn = lr_fn
        self.batch_time_meter = meter_fn
        self.f_nfe_meter = meter_fn
        self.b_nfe_meter = meter_fn
    
    def run(self, epochs, train_dataloader, val_dataloader, test_dataloader):
        train_size = train_dataloader.size
        batches_per_eoch = train_size // train_dataloader.batchsize
        end = time.time()
        for itr in range(epochs):
            for batch_turn in range(batches_per_epoch):
                x, y = train_dataloader.next()
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = self.lr_fn(itr * batches_per_epoch + batch_turn)
                self.optimizer.zero_grad()
                x = x.to(DEVICE)
                y = y.to(DEVICE)
                logits = self.model(x)
                result = logits.cpu().detach().numpy()
                loss = self.criterion(logits, y)
                #print("[%d] Completed loss" % batch_turn)
                nfe_forward = self.model.feature_layers[0].nfe
                self.model.feature_layers[0].nfe = 0

                loss.backward()
                self.optimizer.step()
                #print("[%d] Completed backward" % batch_turn)
                nfe_backward = self.model.feature_layers[0].nfe
                self.model.feature_layers[0].nfe = 0
                self.batch_time_meter.update(time.time() - end)
                self.f_nfe_meter.update(nfe_forward)
                self.b_nfe_meter.update(nfe_backward)
                end = time.time()
                print("[%d] Completed one batch" % batch_turn)

                #with train_summary_writer.as_default():
                    #tensorflow.summary.scalar('loss', loss.item(), step=globaliter)

            with torch.no_grad():
                train_acc = self.accuracy(self.model, test_dataloader)
                val_acc = self.accuracy(self.model, val_dataloader)
                print(
                     "Epoch {:04d} | Time {:.3f} ({:.3f}) | NFE-F {:.1f} | NFE-B {:.1f} | Train Acc {:.4f} | Test Acc {:.4f}".format( itr // batches_per_epoch, batch_time_meter.val, batch_time_meter.avg, f_nfe_meter.avg, b_nfe_meter.avg, train_acc, val_acc )
                     )

#Data Generation

In [0]:
train_dataloader, test_dataloader, val_dataloader = train_val_test_split(batch_size=BATCH_SIZE, validation_ratio=0.1, test_ratio=0.1)

#Initialization

In [0]:
model = ODEModel(9) #method is None
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
criterion = nn.MSELoss().to(DEVICE)
batches_per_epoch = train_dataloader.size // train_dataloader.batchsize
lr_fn = learning_rate_with_decay(
        BATCH_SIZE, batch_denom=128, batches_per_epoch=batches_per_epoch, boundary_epochs=[60, 100, 140],
        decay_rates=[1, 0.1, 0.01, 0.001]
)
trainer = Trainer(model=model,optimizer=optimizer,criterion=criterion,
                  accuracy=accuracy,lr_fn=lr_fn,meter_fn=RunningAverageMeter())

# Train

In [0]:
#%tensorboard --logdir logs/tensorboard

trainer.run(NEPOCHS, train_dataloader, val_dataloader, test_dataloader)

[0] Completed one batch
[1] Completed one batch


# Predict input

adjoint.py의 81번째 코드 참고!

In [0]:
# predict x by y
search_size = 1
for itr in range(min(train_size, search_size)):
    x,y = train_dataloader.next(1)
    y = np.transpose(y.numpy())

    # Linear
    weight = model[5].weight
    bias = model[5].bias.cpu().detach().numpy()
    bias = bias.reshape((2,1))
    z_tn = np.subtract(y, bias)
    inverse_weight = np.linalg.pinv(weight.cpu().detach().numpy())
    z_tn = np.matmul(inverse_weight, z_tn)

    # AdaptiveAvgPool1d(1)
    z_tn = np.repeat(z_tn, EMBED_SIZE, axis=1)
    z_tn = np.expand_dims(z_tn, axis=0)

    # ode back
    z_tn = torch.from_numpy(z_tn).to(DEVICE)
    z_t0 = odeint(ODEf, z_tn, torch.tensor([1, 0]).float().type_as(z_tn) , rtol=RTOL, atol=ATOL, method = 'adams')[1]
    # output[0] is for time t0, output[1] is for time t1.

    assert z_t0.shape == (1, 9,18)

    z_t0 = z_t0.cpu().detach().numpy()
    encoded = max_index(z_t0)
    
    print("encoded")
    for enc_ in encoded:
        print(enc_)
    
    mat, op = decode(encoded)

    print("mat")
    for mat_ in mat:
        print(mat_)
    print("y:", y)


    def __run_experiment(mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench_api.query(model_spec)
        return q['validation_accuracy'], q['training_time']


    print("result", __run_experiment(mat, op))